In [14]:
# 导入所需的库
import rosbag

# 定义回调函数
def bag_callback(topic, msg, t):
    print(f"Received message from topic {topic}: {msg}")

# 打开.bag文件
bag = rosbag.Bag('/space0/imagelab/need_label/2023-06-20-05-50-35.bag')
print(bag.get_type_and_topic_info().topics)
# 读取.bag文件中的消息
# for topic, msg, t in bag.read_messages():
#     bag_callback(topic, msg, t)
#     break
# print(bag.read_messages())
# 关闭.bag文件
bag.close()

{'/camera/color/image_raw': TopicTuple(msg_type='sensor_msgs/Image', message_count=384, connections=1, frequency=29.956960524530217), '/ir/image_raw': TopicTuple(msg_type='sensor_msgs/Image', message_count=384, connections=1, frequency=29.957174487536605), '/livox/imu': TopicTuple(msg_type='sensor_msgs/Imu', message_count=2594, connections=1, frequency=33554.432), '/livox/lidar': TopicTuple(msg_type='sensor_msgs/PointCloud2', message_count=128, connections=1, frequency=9.995338706371864)}


In [27]:
import os
import rosbag
from sensor_msgs.msg import Image, Imu, PointCloud2
from cv_bridge import CvBridge
import cv2

# 定义输入 .bag 文件所在文件夹路径
input_folder = '/space0/imagelab/need_label'

# 定义输出文件夹的根目录
output_folder = 'output_data'
os.makedirs(output_folder, exist_ok=True)

# 遍历 .bag 文件所在文件夹并解析数据
bag_files = [f for f in os.listdir(input_folder) if f.endswith('.bag')]

for bag_file in bag_files:
    bag_path = os.path.join(input_folder, bag_file)
    bag = rosbag.Bag(bag_path)
    print(f"正在处理 {bag_path} 文件...")
    print(bag.get_type_and_topic_info().topics)
    bag.close()
print("所有 .bag 文件的数据已成功整理到各个文件夹中。")


正在处理 /space0/imagelab/need_label/2023-06-20-06-14-58.bag 文件...
{'/camera/color/image_raw': TopicTuple(msg_type='sensor_msgs/Image', message_count=7911, connections=1, frequency=29.99198412556535), '/ir/image_raw': TopicTuple(msg_type='sensor_msgs/Image', message_count=7913, connections=1, frequency=29.98994687430733), '/livox/imu': TopicTuple(msg_type='sensor_msgs/Imu', message_count=53492, connections=1, frequency=35848.75213675214), '/livox/lidar': TopicTuple(msg_type='sensor_msgs/PointCloud2', message_count=2638, connections=1, frequency=9.999008277072129)}
正在处理 /space0/imagelab/need_label/2023-06-20-05-50-35.bag 文件...
{'/camera/color/image_raw': TopicTuple(msg_type='sensor_msgs/Image', message_count=384, connections=1, frequency=29.956960524530217), '/ir/image_raw': TopicTuple(msg_type='sensor_msgs/Image', message_count=384, connections=1, frequency=29.957174487536605), '/livox/imu': TopicTuple(msg_type='sensor_msgs/Imu', message_count=2594, connections=1, frequency=33554.432), '/l

In [33]:
import os

def count_files_in_subdirectories(directory):
    # 创建一个字典用于存储子文件夹及其文件数目
    folder_file_count = {}

    # 遍历指定目录下的所有子文件夹
    for root, dirs, files in os.walk(directory):
        # 获取当前子文件夹的路径
        folder_path = os.path.abspath(root)

        # 统计当前子文件夹中的文件数目
        file_count = len(files)

        # 将子文件夹及其文件数目存储到字典中
        folder_file_count[folder_path] = file_count

    return folder_file_count

# 请将 'data' 替换为你要统计的目录名
directory_to_count = '/space0/imagelab/need_label/output_data'

# 调用函数统计目录下各子文件夹的文件数目
result = count_files_in_subdirectories(directory_to_count)

# 打印结果
for folder, count in result.items():
    print(f"子文件夹 '{folder}' 中的文件数目：{count}")


子文件夹 '/space0/imagelab/need_label/output_data' 中的文件数目：0
子文件夹 '/space0/imagelab/need_label/output_data/camera_color_image_raw' 中的文件数目：19226
子文件夹 '/space0/imagelab/need_label/output_data/ir_image_raw' 中的文件数目：19237
子文件夹 '/space0/imagelab/need_label/output_data/livox_lidar' 中的文件数目：6413
子文件夹 '/space0/imagelab/need_label/output_data/livox_imu' 中的文件数目：129998


In [ ]:
import rosbag
import os
from sensor_msgs.msg import Image, Imu, PointCloud2
from cv_bridge import CvBridge
import cv2

# 定义 .bag 文件路径
bag_file = '/space0/imagelab/need_label/2023-06-20-05-50-35.bag'

# 定义输出文件夹的根目录
output_folder = 'data'
os.makedirs(output_folder, exist_ok=True)

# 读取 .bag 文件
bag = rosbag.Bag(bag_file)

# 初始化 CvBridge
cv_bridge = CvBridge()

# 定义函数用于保存 Image 类型的数据到文件夹中
def save_image_msg(msg, folder):
    cv_image = cv_bridge.imgmsg_to_cv2(msg, desired_encoding='passthrough')
    file_name = os.path.join(folder, f"{msg.header.stamp}.jpg")
    cv2.imwrite(file_name, cv_image)

# 定义函数用于保存 Imu 类型的数据到文件夹中
def save_imu_msg(msg, folder):
    file_name = os.path.join(folder, f"{msg.header.stamp}.txt")
    with open(file_name, 'w') as f:
        f.write(f"Linear Acceleration: x={msg.linear_acceleration.x}, y={msg.linear_acceleration.y}, z={msg.linear_acceleration.z}\n")
        f.write(f"Angular Velocity: x={msg.angular_velocity.x}, y={msg.angular_velocity.y}, z={msg.angular_velocity.z}\n")

# 定义函数用于保存 PointCloud2 类型的数据到文件夹中
def save_lidar_msg(msg, folder):
    lidar_data = msg.data
    file_name = os.path.join(folder, f"{msg.header.stamp}.bin")
    with open(file_name, 'wb') as f:
        f.write(lidar_data)

# 解析并保存数据到各自的文件夹
for topic, topic_info in bag.get_type_and_topic_info().topics.items():
    topic_fields = topic.strip('/').split('/')
    folder_name = '_'.join(topic_fields)
    folder = os.path.join(output_folder, folder_name)
    os.makedirs(folder, exist_ok=True)

    for _, msg, _ in bag.read_messages(topic):
        if topic_info.msg_type == 'sensor_msgs/Image':
            save_image_msg(msg, folder)
        elif topic_info.msg_type == 'sensor_msgs/Imu':
            save_imu_msg(msg, folder)
        elif topic_info.msg_type == 'sensor_msgs/PointCloud2':
            save_lidar_msg(msg, folder)
        else:
            # Handle other message types if needed
            pass

bag.close()

print("数据已成功整理到各个文件夹中。")

In [37]:
import os
import rosbag
from sensor_msgs.msg import Image, Imu, PointCloud2
from cv_bridge import CvBridge
import cv2
import json

# 定义输入 .bag 文件所在文件夹路径
input_folder = '/space0/imagelab/need_label'

# 定义输出文件夹的根目录
output_folder = 'output_data'
os.makedirs(output_folder, exist_ok=True)

# 初始化 CvBridge
cv_bridge = CvBridge()

# 定义函数用于保存 Image 类型的数据到文件夹中
# def save_image_msg(msg, folder):
#     cv_image = cv_bridge.imgmsg_to_cv2(msg, desired_encoding='passthrough')
#     file_name = os.path.join(folder, f"{msg.header.stamp}.jpg")
#     cv2.imwrite(file_name, cv_image)
def save_image_msg(msg, folder):
    image = cv_bridge.imgmsg_to_cv2(msg, desired_encoding='passthrough')

    # 创建文件名
    image_file = os.path.join(folder, f"{msg.header.stamp}.png")

    # 保存图像数据到文件
    cv2.imwrite(image_file, image)

    # 创建并保存格式信息到 JSON 文件
    format_info = {
        'height': msg.height,
        'width': msg.width,
        'encoding': msg.encoding,
        'is_bigendian': msg.is_bigendian,
        'step': msg.step,
    }
    json_file = os.path.join(folder, f"{msg.header.stamp}.json")
    with open(json_file, 'w') as f:
        json.dump(format_info, f)

# # 定义函数用于保存 Imu 类型的数据到文件夹中
# def save_imu_msg(msg, folder):
#     file_name = os.path.join(folder, f"{msg.header.stamp}.txt")
#     with open(file_name, 'w') as f:
#         f.write(f"Linear Acceleration: x={msg.linear_acceleration.x}, y={msg.linear_acceleration.y}, z={msg.linear_acceleration.z}\n")
#         f.write(f"Angular Velocity: x={msg.angular_velocity.x}, y={msg.angular_velocity.y}, z={msg.angular_velocity.z}\n")

# # 定义函数用于保存 PointCloud2 类型的数据到文件夹中
# def save_lidar_msg(msg, folder):
#     lidar_data = msg.data
#     file_name = os.path.join(folder, f"{msg.header.stamp}.bin")
#     with open(file_name, 'wb') as f:
#         f.write(lidar_data)

def save_imu_msg(msg, folder):
    imu_data = {
        'header': {
            'seq': msg.header.seq,
            'stamp': {
                'secs': msg.header.stamp.secs,
                'nsecs': msg.header.stamp.nsecs,
            },
            'frame_id': msg.header.frame_id,
        },
        'orientation': {
            'x': msg.orientation.x,
            'y': msg.orientation.y,
            'z': msg.orientation.z,
            'w': msg.orientation.w,
        },
        'orientation_covariance': msg.orientation_covariance,
        'angular_velocity': {
            'x': msg.angular_velocity.x,
            'y': msg.angular_velocity.y,
            'z': msg.angular_velocity.z,
        },
        'angular_velocity_covariance': msg.angular_velocity_covariance,
        'linear_acceleration': {
            'x': msg.linear_acceleration.x,
            'y': msg.linear_acceleration.y,
            'z': msg.linear_acceleration.z,
        },
        'linear_acceleration_covariance': msg.linear_acceleration_covariance,
    }
    file_name = os.path.join(folder, f"{msg.header.stamp}.json")
    with open(file_name, 'w') as f:
        json.dump(imu_data, f)

def save_lidar_msg(msg, folder):
    # 提取 PointCloud2 的元数据
    header = msg.header
    height = msg.height
    width = msg.width
    fields = msg.fields
    is_bigendian = msg.is_bigendian
    point_step = msg.point_step
    row_step = msg.row_step
    data = msg.data

    # 创建文件名
    json_file = os.path.join(folder, f"{header.stamp}.json")
    bin_file = os.path.join(folder, f"{header.stamp}.bin")

    # 保存 PointCloud2 的元数据到 JSON 文件
    metadata = {
        'header': {
            'seq': header.seq,
            'stamp': {
                'secs': header.stamp.secs,
                'nsecs': header.stamp.nsecs,
            },
            'frame_id': header.frame_id,
        },
        'height': height,
        'width': width,
        'fields': [],
        'is_bigendian': is_bigendian,
        'point_step': point_step,
        'row_step': row_step,
    }

    for field in fields:
        metadata['fields'].append({
            'name': field.name,
            'offset': field.offset,
            'datatype': field.datatype,
            'count': field.count,
        })

    with open(json_file, 'w') as f:
        json.dump(metadata, f, indent=4)

    # 保存二进制数据到文件
    with open(bin_file, 'wb') as f:
        f.write(data)


# 遍历 .bag 文件所在文件夹并解析数据
bag_files = [f for f in os.listdir(input_folder) if f.endswith('.bag')]

for bag_file in bag_files:
    bag_path = os.path.join(input_folder, bag_file)
    bag = rosbag.Bag(bag_path)

    for topic, topic_info in bag.get_type_and_topic_info().topics.items():
        topic_fields = topic.strip('/').split('/')
        folder_name = '_'.join(topic_fields)
        folder = os.path.join(output_folder, folder_name)
        os.makedirs(folder, exist_ok=True)

        for _, msg, _ in bag.read_messages(topic):
            if topic_info.msg_type == 'sensor_msgs/Image':
                save_image_msg(msg, folder)
                # pass
            elif topic_info.msg_type == 'sensor_msgs/Imu':
                # save_imu_msg(msg, folder)
                pass
            elif topic_info.msg_type == 'sensor_msgs/PointCloud2':
                # save_lidar_msg(msg, folder)
                pass
            else:
                # Handle other message types if needed
                pass

    bag.close()

print("所有 .bag 文件的数据已成功整理到各个文件夹中。")


所有 .bag 文件的数据已成功整理到各个文件夹中。


In [38]:
import os

def count_files_in_subdirectories(directory):
    # 创建一个字典用于存储子文件夹及其文件数目
    folder_file_count = {}

    # 遍历指定目录下的所有子文件夹
    for root, dirs, files in os.walk(directory):
        # 获取当前子文件夹的路径
        folder_path = os.path.abspath(root)

        # 初始化文件类型计数器
        png_count = 0
        json_count = 0
        bin_count = 0

        # 统计当前子文件夹中各文件类型的数量
        for file in files:
            if file.endswith('.png'):
                png_count += 1
            elif file.endswith('.json'):
                json_count += 1
            elif file.endswith('.bin'):
                bin_count += 1

        # 将子文件夹及其文件数目存储到字典中
        folder_file_count[folder_path] = {
            'png': png_count,
            'json': json_count,
            'bin': bin_count
        }

    return folder_file_count

# 请将 'data' 替换为你要统计的目录名
directory_to_count = '/space0/imagelab/need_label/output_data'

# 调用函数统计目录下各子文件夹的文件数目
result = count_files_in_subdirectories(directory_to_count)

# 打印结果
for folder, file_counts in result.items():
    print(f"子文件夹 '{folder}' 中的文件数目：")
    print(f"PNG 文件数目：{file_counts['png']}")
    print(f"JSON 文件数目：{file_counts['json']}")
    print(f"BIN 文件数目：{file_counts['bin']}")
    print()


子文件夹 '/space0/imagelab/need_label/output_data' 中的文件数目：
PNG 文件数目：0
JSON 文件数目：0
BIN 文件数目：0

子文件夹 '/space0/imagelab/need_label/output_data/camera_color_image_raw' 中的文件数目：
PNG 文件数目：19226
JSON 文件数目：19226
BIN 文件数目：0

子文件夹 '/space0/imagelab/need_label/output_data/ir_image_raw' 中的文件数目：
PNG 文件数目：19237
JSON 文件数目：19237
BIN 文件数目：0

子文件夹 '/space0/imagelab/need_label/output_data/livox_lidar' 中的文件数目：
PNG 文件数目：0
JSON 文件数目：6413
BIN 文件数目：6413

子文件夹 '/space0/imagelab/need_label/output_data/livox_imu' 中的文件数目：
PNG 文件数目：0
JSON 文件数目：129998
BIN 文件数目：0

